### Receipt Parsing

In [1]:
! pip install langchain
!pip install openai

In [11]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from decimal import Decimal
import json
import os
import csv

# api_key = ""
model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=api_key)

In [12]:
prompt_examples = [
  {
        "receipt_text": """Marley's Shop\n123 Long Rd\nKailua, HI 67530\n(808) 555-1234\nCASHIER: JOHN\nREGISTER #: 6\n04/12/2023\nTransaction ID: 5769009\nPRICE QTY TOTAL\nAPPLES (1 lb)\n2.99 2 5.98 1001\n-1.00 999\nChoco Dream Cookies\n7.59 1 7.59 1001\nSUBTOTAL\n13.57\nSALES TAX 8.5%\n1.15\nTOTAL\n-14.72\nVISA CARD 14.72\nCARD#: ************1234\nREFERENCE#: 6789\nTHANK YOU FOR SHOPPING WITH US!""",
        "receipt_info": """
{{
  "ReceiptInfo": {{
    "merchant": "Marley's Shop",
    "address": "123 Long Rd",
    "city": "Kailua",
    "state": "HI",
    "phoneNumber": "(808) 555-1234",
    "tax": 1.15,
    "total": 14.72,
    "receiptDate": "04/12/2023",
    "ITEMS": [
      {{
        "description": "APPLES (1 lb)",
        "quantity": 2,
        "unitPrice": 2.99,
        "totalPrice": 5.98,
        "discountAmount": 1.00
      }},
      {{
        "description": "Choco Dream Cookies",
        "quantity": 1,
        "unitPrice": 7.59,
        "totalPrice": 7.59
      }}
    ]
  }}
}}
"""
    }
#     ,
#      {
#         "receipt_text": """203\nBUY ONE GET ONE FREE QUARTER POUNDER\nW/CHEESE OR EGG MCMUFFIN\nGo to www.mcdvoice.com within 7 days\nand tell <UNKNOWN> about your visit.\n...\n08/19/2023 03:33 PM\n...\n1 Happy Meal Ch Burger\n5.89\n1 Cheeseburger\nONLY Onions\nONLY Ketchup\n1 Extra Kids Fry\n1 Apple Juice\n1 NERF\nSubtotal\n5.89\nTax\n0.28\nTake-Out Total\n6.17\nCashless\n6.17\nChange\n0.00\nMER# 467782\n...""",
#         "receipt_info": """
# {{
#   "ReceiptInfo": {{
#     "merchant": "McDonald's",
#     "address": "2915 E Manca Rd",
#     "city": "<UNKNOWN>",
#     "state": "<UNKNOWN>",
#     "phoneNumber": "<UNKNOWN>",
#     "tax": 0.28,
#     "total": 6.17,
#     "receiptDate": "08/19/2023",
#     "receiptTime": "03:33 PM",
#     "ITEMS": [
#       {{
#         "description": "Happy Meal Ch Burger",
#         "quantity": 1,
#         "unitPrice": 5.89,
#         "totalPrice": 5.89,
#         "discountAmount": 0.00
#       }}
#     ]
#   }}
# }}
# """
#     },
#     {
#     "receipt_text": """H MART\n...\n458 Keawe st\nHonolulu, <UNKNOWN> 96813\n...\n06/11/23 09:15pm\nWANG DANGMYUN\n4.99 B\nK-T TOPPOKI\nPC\n6.99 B\nOTG SOUP RICECAKE\n7.99 B\nHT VEGE DUMPLING\n14.99 B\nTAX\n1.65\n****\nBALANCE\n36.61\nDiscover Credit - C\n...\nTOTAL AMOUNT: $36.61""",
#     "receipt_info": """
# {{
#   "ReceiptInfo": {{
#     "merchant": "H MART",
#     "address": "458 Keawe st",
#     "city": "Honolulu",
#     "state": "<UNKNOWN>",
#     "phoneNumber": "(808) 219-0924",
#     "tax": 1.65,
#     "total": 36.61,
#     "receiptDate": "06/11/23",
#     "receiptTime": "09:15 PM",
#     "ITEMS": [
#       {{
#         "description": "WANG DANGMYUN",
#         "quantity": 1,
#         "unitPrice": 4.99,
#         "totalPrice": 4.99,
#         "discountAmount": 0.00
#       }},
#       {{
#         "description": "K-T TOPPOKI",
#         "quantity": 1,
#         "unitPrice": 6.99,
#         "totalPrice": 6.99,
#         "discountAmount": 0.00
#       }},
#       {{}
#          "description": "OTG SOUP RICECAKE",
#          "quantity": 1,
#          "unitPrice": 7.99,
#          "totalPrice": 7.99,
#          "discountAmount": 0.00
#       }},
#       {{
#           "description": "HT VEGE DUMPLING",
#           "quantity": 1,
#           "unitPrice": 14.99,
#           "totalPrice": 14.99,
#           "discountAmount": 0.00
#       }}
#     ]
#   }}
# }}
# """
#     },

# {
#     "receipt_text": """SAFEWAY\n...\n2855 East Manoa Road\nHONOLULU HI 96822\n...\n08/01/23 20:15\nGROCERY\n6.99 B\nMARINARA SCE\n...\nREFRIG/FROZEN\nLUC CHES COLBY JAC.\n10.99 B\n...\nMARIE CALLENDER'S\n4.00 B\n...\nMEAT\nAIDELLS MEATBALLS\n5.99 B\n...\nTAX\n1.32\n**** BALANCE\n29.29\nCredit Purchase 08/01/23 20:15\n...""",
#      "receipt_info": """
# {{
#   "ReceiptInfo": {{
#     "merchant": "SAFEWAY",
#     "address": "2855 East Manoa Road",
#     "city": "HONOLULU",
#     "state": "HI",
#     "phoneNumber": "(808) 988-2058",
#     "tax": 1.32,
#     "total": 29.29,
#     "receiptDate": "08/01/23",
#     "receiptTime": "20:15",
#     "ITEMS": [
#       {{
#         "description": "MARINARA SCE 6.99 B",
#         "quantity": 1,
#         "unitPrice": 6.99,
#         "totalPrice": 6.99,
#         "discountAmount": 1.50
#       }},
#       {{
#         "description": "LUC CHES COLBY JAC. 10.99 B",
#         "quantity": 1,
#         "unitPrice": 10.99,
#         "totalPrice": 10.99,
#         "discountAmount": 2.50
#       }},
#       {{
#         "description": "MARIE CALLENDER'S 4.00 B",
#         "quantity": 1,
#         "unitPrice": 4.00,
#         "totalPrice": 4.00,
#         "discountAmount": 2.49
#       }},
#       {{
#         "description": "AIDELLS MEATBALLS 5.99 B",
#         "quantity": 1,
#         "unitPrice": 5.99,
#         "totalPrice": 5.99,
#         "discountAmount": 3.00
#       }}
#     ]
#   }}
# }}
# """
# }}
]



example_prompt_str =  """
receipt text:
{receipt_text}
receipt info:
{receipt_info}"""

example_prompt = PromptTemplate(
    input_variables=["receipt_text", "receipt_info"],
    template=example_prompt_str
)

your_receipt_example = prompt_examples[0]

data = {"receipt_text": your_receipt_example["receipt_text"],"receipt_info": your_receipt_example["receipt_info"]}
print(example_prompt.format(**data))


receipt text:
Marley's Shop
123 Long Rd
Kailua, HI 67530
(808) 555-1234
CASHIER: JOHN
REGISTER #: 6
04/12/2023
Transaction ID: 5769009
PRICE QTY TOTAL
APPLES (1 lb)
2.99 2 5.98 1001
-1.00 999
Choco Dream Cookies
7.59 1 7.59 1001
SUBTOTAL
13.57
SALES TAX 8.5%
1.15
TOTAL
-14.72
VISA CARD 14.72
CARD#: ************1234
REFERENCE#: 6789
THANK YOU FOR SHOPPING WITH US!
receipt info:

{{
  "ReceiptInfo": {{
    "merchant": "Marley's Shop",
    "address": "123 Long Rd",
    "city": "Kailua",
    "state": "HI",
    "phoneNumber": "(808) 555-1234",
    "tax": 1.15,
    "total": 14.72,
    "receiptDate": "04/12/2023",
    "ITEMS": [
      {{
        "description": "APPLES (1 lb)",
        "quantity": 2,
        "unitPrice": 2.99,
        "totalPrice": 5.98,
        "discountAmount": 1.00
      }},
      {{
        "description": "Choco Dream Cookies",
        "quantity": 1,
        "unitPrice": 7.59,
        "totalPrice": 7.59
      }}
    ]
  }}
}}



In [13]:
class Item(BaseModel):
    description: str = Field(..., description="Item description")
    quantity: int = Field(..., description="Quantity of the item")
    unitPrice: Decimal = Field(..., description="Price per unit")
    totalPrice: Decimal = Field(..., description="Total price for the item")
    discountAmount: Decimal = Field(..., description="Discount amount on the item")

class ReceiptInfo(BaseModel):
    merchant: str = Field(..., description="Merchant name")
    address: str = Field(..., description="Merchant address")
    city: str = Field(..., description="City")
    state: str = Field(..., description="State")
    phoneNumber: str = Field(..., description="Merchant phone number")
    tax: Decimal = Field(..., description="Tax amount")
    total: Decimal = Field(..., description="Total amount")
    receiptDate: str = Field(..., description="Date of the receipt")
    receiptTime: str = Field(..., description="Time of the receipt")
    ITEMS: list[Item] = Field(..., description="List of items purchased")

receiptParser = PydanticOutputParser(pydantic_object=ReceiptInfo)

prompt_prefix = """Extract information from the provided receipt text and format it into a structured JSON.
Try to retrieve as much data as possible from the receipt. For fields where specific information is not available or cannot be extracted, use <UNKNOWN> as the value.

{format_instructions}

"""
# For any field where information is not available or cannot be extracted from the receipt, use '<UNKNOWN>' as the value for that field.


# print(receiptParser.get_format_instructions())

execute_fewshot_prompt = FewShotPromptTemplate(
    prefix=prompt_prefix,
    input_variables=["receipt_text"],
    partial_variables={"format_instructions": receiptParser.get_format_instructions()},
    examples=prompt_examples,  # Define examples of receipts and their structured output
    example_prompt=example_prompt,  # Define a prompt that guides the model on how to format the receipt
    example_separator="\n\n",
    suffix="Receipt text: {receipt_text}\n"
)

# Example receipt text
receipt_text =  """SAFEWAY\n...\n2855 East Manoa Road\nHONOLULU HI 96822\n...\n08/01/23 20:15\nGROCERY\n6.99 B\nMARINARA SCE\n...\nREFRIG/FROZEN\nLUC CHES COLBY JAC.\n10.99 B\n...\nMARIE CALLENDER'S\n4.00 B\n...\nMEAT\nAIDELLS MEATBALLS\n5.99 B\n...\nTAX\n1.32\n**** BALANCE\n29.29\nCredit Purchase 08/01/23 20:15\n...""",

data = {"receipt_text": receipt_text}

# chain = execute_fewshot_prompt | model | receiptParser
chain = execute_fewshot_prompt | model
response = chain.invoke(data)
# response = chain.invoke(data, "temperature": 0.7,"max_tokens": 150)
print(response.content)

{
  "ReceiptInfo": {
    "merchant": "SAFEWAY",
    "address": "2855 East Manoa Road",
    "city": "HONOLULU",
    "state": "HI",
    "phoneNumber": "<UNKNOWN>",
    "tax": 1.32,
    "total": 29.29,
    "receiptDate": "08/01/23",
    "ITEMS": [
      {
        "description": "MARINARA SCE",
        "quantity": 1,
        "unitPrice": 6.99,
        "totalPrice": 6.99
      },
      {
        "description": "LUC CHES COLBY JAC.",
        "quantity": 1,
        "unitPrice": 10.99,
        "totalPrice": 10.99
      },
      {
        "description": "MARIE CALLENDER'S",
        "quantity": 1,
        "unitPrice": 4.00,
        "totalPrice": 4.00
      },
      {
        "description": "AIDELLS MEATBALLS",
        "quantity": 1,
        "unitPrice": 5.99,
        "totalPrice": 5.99
      }
    ]
  }
}


In [22]:
import os
def read_text_files(folder_path):
    files_content = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r') as file:
                files_content.append(file.read())
    return files_content

# Folder containing receipt text files
folder_path = 'Receipts'

receipt_texts = read_text_files(folder_path)
# receipt_data = []

# Process each receipt text
for idx, receipt_text in enumerate(receipt_texts):
    data = {"receipt_text": receipt_text}
    
    chain = execute_fewshot_prompt | model
    response = chain.invoke(data)
    # response = chain.invoke(data, "temperature": 0.7,"max_tokens": 150)
    print("Receipt Text:\n",receipt_text)
    print("Receipt Info:\n",response.content )
    receipt_data.append([receipt_text, response.content])
#     break

# Write data to CSV
with open('receipts_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    # Write header
    csvwriter.writerow(['Receipt Text', 'Receipt Info'])

    # Write receipt data
    for row in receipt_data:
        csvwriter.writerow(row)

print('CSV file created with receipt data.')


["<UNKNOWN>\nTEA <UNKNOWN>\n<UNKNOWN>\nStore <UNKNOWN> - MFNOA\nStore <UNKNOWN> Benjamin Camello\n<UNKNOWN>\n2754 <UNKNOWN> Drive, <UNKNOWN> 7-106\n<UNKNOWN> HI <UNKNOWN>\nL <UNKNOWN> LIGHT\n3.09 T\nTAX\n0.15\n**** BALANCE\n3.24\nSTORE#: 3001\nTYPE: Purchase\nVISA CREDIT\nCARD#: <UNKNOWN> t\nAID: <UNKNOWN>\nAPP CODE: <UNKNOWN> SEQ#: 50561\nREFERENCE#: 35000043\nTERM#: 50 <UNKNOWN> 775\nDATE: 05/04/22 TIME: <UNKNOWN>\nAPPROVED - Purchase\nAMOUNT: $3.24\nMERCHANT ID: 91'68716\nAC: <UNKNOWN> <UNKNOWN> 80\nTVR: 0000000000\nTSI: 0000\nVisa\n3.24\n<UNKNOWN>\n0.00\nTOTAL NUMBER OF ITEMS SOLD =\n1\n05/04/22 <UNKNOWN> 3001 50 6061 775\n<UNKNOWN> TYLER\nVisit Our Website\n<UNKNOWN>\nus out on social media:\n<UNKNOWN>\n<UNKNOWN>\n<UNKNOWN>\nRefunds <UNKNOWN> be issued based on the\noriginal <UNKNOWN> <UNKNOWN> payment on receipt.\nStore <UNKNOWN> <UNKNOWN> will be issued for\ngifts or without a receipt.\nORDER #6061\n", '{\n  "ReceiptInfo": {\n    "merchant": "Store <UNKNOWN>",\n    "address": "2

In [14]:
import json
import pandas as pd

# Example list of JSON strings
json_strings = [response.content]  # Your list of JSON strings

# Function to parse a single JSON string and extract data
def parse_json(json_str):
    data = json.loads(json_str)
    receipt_info = data["ReceiptInfo"]
    
    # Extracting basic information
    basic_info = {k: receipt_info[k] for k in receipt_info if k != "ITEMS"}

    # Handling ITEMS
    items = receipt_info["ITEMS"]
    for i, item in enumerate(items, 1):
        for key, value in item.items():
            basic_info[f"item{i}_{key}"] = value
    
    return basic_info

# Parsing and collecting data
parsed_data = [parse_json(js) for js in json_strings]

# Creating DataFrame
df = pd.DataFrame(parsed_data)

# Save the DataFrame to an Excel file
excel_filename = 'receipt_data.xlsx'
df.to_excel(excel_filename, index=False)


In [2]:
import pandas as pd
import numpy as np
import gensim.downloader as api
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation
import faiss

# Step 1: Read the CSV file
df = pd.read_excel('All_Categories.xlsx')  # Replace with your file path

def preprocess(text):
#     print("Before",text)
    text = text.lower()
    text = strip_punctuation(text)
    text = remove_stopwords(text)
#     print("After", text)
    return text.split()

df['processed_descriptions'] = df['Category Description'].apply(preprocess)

In [3]:
model = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [10]:
# model = api.load("word2vec-google-news-300")
# model = api.load("glove-twitter-100")
# model = api.load("glove-wiki-gigaword-100")
# model = api.load("fasttext-wiki-news-subwords-300")

def get_embedding(words):
    valid_words = [word for word in words if word in model.key_to_index]
    if valid_words:
        return np.mean([model[word] for word in valid_words], axis=0)
    else:
        return np.zeros(100)  # 300 is the dimension of Word2Vec vectors


df['embeddings'] = df['processed_descriptions'].apply(get_embedding)

faiss_index = faiss.read_index("faiss_index.index")

dim = 100  # Dimension of Word2Vec vectors
# faiss_index = faiss.IndexFlatL2(dim)
# faiss_index.add(np.array(df['embeddings'].tolist()).astype('float32'))

# faiss.write_index(faiss_index, "faiss_index.index")


def categorize_receipt(receipt_description):
    processed = preprocess(receipt_description)
    embedding = get_embedding(processed).reshape(1, -1).astype('float32')
    distances, indices = faiss_index.search(embedding, 1)
    return df.iloc[indices[0][0]]['Category Name']

receipt_description = " soccer cleats track jackets leggings backpacks"
category = categorize_receipt(receipt_description)
print("Category:", category)


Category: Clothing and Apparel
